In [70]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.preprocessing import StandardScaler
import math
import plotly.io as pio

In [71]:
# Save name of mouse we are graphing
mouse = input("What mouse are you graphing?\t")
print(mouse)

FN13_20240618


In [72]:
# Load in all data necessary for graphing
area_per_frame = pd.read_csv("../pupil_data_analysis/FN13_js_self/area_per_frame_no_outliers.csv").iloc[:, 1]
camlog_times = pd.read_excel("camlog.xlsx")
trial_times = pd.read_csv("trial_times.csv")
# print(area_per_frame.head())
# print(camlog_times.head())
# print(trial_times.head())

In [73]:
trials = []
trial = []
trial_num = 0
inTrial = False
startTime = camlog_times.iloc[0]

for index, camlog in camlog_times.iterrows():
    # Check trial_num overflow before proceeding
    if trial_num >= len(trial_times):
        # print(f"Trial number {trial_num} exceeded bounds. Exiting.")
        break

    # Compute time delta between camlog and startTime
    delta = (camlog - startTime).iloc[0] if isinstance(camlog - startTime, pd.Series) else camlog - startTime
    # print(delta)

    # End trial if the current time exceeds trial end time
    if delta > trial_times.iloc[trial_num, 2] and inTrial:
        # print(f"Trial {trial_num + 1} End: {delta}")
        trials.append(trial)
        inTrial = False
        trial_num += 1
        # print(f"Moving to trial {trial_num}")
        
        # Re-check trial_num bounds after incrementing
        if trial_num >= len(trial_times):
            # print(f"Trial number {trial_num} exceeded bounds after increment. Exiting.")
            break

    # Start a new trial if the current time exceeds trial start time and not already in trial
    elif delta > trial_times.iloc[trial_num, 1] and not inTrial:
        # print(f"Trial {trial_num + 1} Start: {delta}")
        trial = []
        inTrial = True

    # Always append index to trial if in a trial
    if inTrial:
        trial.append(index)

print(f"Total trials detected: {len(trials)}")


Total trials detected: 164


In [74]:
pio.renderers.default = 'browser'  # This will open the plots in your default web browser

pages = 2
num_graphs = len(trials)
for index in range(pages):
    # Creates each subplot and creates a placeholder subplot title
    fig = make_subplots(
        rows=int(np.ceil(num_graphs / pages)), 
        cols=1, 
        subplot_titles=[f"temp_subtitle {int(date)}" for date in range(int(num_graphs))]
    )
    # Creates pupil area subplot graphs in each graphset
    start = index * int(num_graphs / pages)
    end = min((index + 1) * int(num_graphs / pages), len(trials))
    for i in range(start, end):
        fig.append_trace(go.Scatter(
            x=trials[i],
            y=area_per_frame.iloc[trials[i][0]:trials[i][-1]],
            showlegend=False,
            line=dict(color='black'),
        ), row=i + 1 - index * int(num_graphs / pages), col=1)

        # Updates all the labels around each of the subplots
        fig.layout.annotations[i - index * int(num_graphs / pages)]['text'] = "Subplot " + str(
            i + 1) + ": Frames " + str(trials[i][0]) + " - " + str(
            trials[i][-1]) + " and Seconds: " + str(
            round(trials[i][0] / 30)) + " - " + str(round(trials[i][-1] / 30))
        fig.update_xaxes(row=i + 1 - index * int(num_graphs / pages),
                         col=1,
                         title_text="Time (Seconds)",
                         showline=True,
                         linewidth=2,
                         linecolor='black',
                         ticks="outside",
                         tickwidth=1,
                         tickcolor='black',
                         ticklen=7)
        fig.update_yaxes(row=i + 1 - index * int(num_graphs / pages),
                         col=1,
                         title_text="Pupil Area (Pixels)",
                         showline=True,
                         linewidth=2,
                         linecolor='black',
                         dtick=100,
                         ticks="outside",
                         tickwidth=1,
                         tickcolor='black',
                         ticklen=7)

    fig.update_layout(title="Graph Set " + str(index + 1) + " " + mouse + " Pupil Area for each Trial",
                      height=250 * 100,
                      width=1250,
                      plot_bgcolor="white")

    # fig.write_html("graph_set_" + str(index + 1) + "_" + mouse + "_pupil_area_per_trial.html")

    # Shows the first 5 Graph Sets to confirm everything is working as intended
    if index < 5:
        fig.show()
    index = index + 1